In [1]:
from digital_manuscript import BnF
manuscript = BnF()

In [2]:
import os
import pandas as pd
import re
import urllib.request
from urllib.error import HTTPError

Setup file paths:

In [3]:
cwd = os.getcwd()
#m_path = cwd if 'manuscript-object' not in cwd else f'{cwd}/../'
m_path = cwd if 'manuscript-object' in cwd else f'{cwd}/manuscript-object/'
fieldnotes_path = f'{m_path}/fieldnotes/FA18+other-fieldnotes-list+links.csv'   # later try to implement pulling directly from Google Sheets
out_path = f'{m_path}/fieldnotes/fieldnotes_refs.csv'


Setup labels and text stuff:

In [4]:
df_url_column = 'full-html'
df_refs_column = 'references'

re_href = re.compile(r'<a href="(.*?)">') # get hyperlinked urls

Script:

In [5]:
fieldnotes = pd.read_csv(fieldnotes_path)

Loop through the urls column:

In [6]:
references = [None] * len(fieldnotes[df_url_column])  # initialize list of proper length filled with falsy values
for i, url in enumerate(fieldnotes[df_url_column]):
    if 'http://' not in url:
        url = f'http://{url}'
        
    try:
        f = urllib.request.urlopen(url)
    except HTTPError:
        print(f"URL not found: {url}")
        references[i] = 'url not found'
        continue
    
    page_text = f.read().decode('utf-8')
    
    matches = re_href.findall(page_text)
    
    for match in matches:
        if 'drive.google.com' in match:
            if not references[i]:
                references[i] = match        # first url
            else:
                references[i] += ',' + match # append next url, separated by comma
    
    # if no references in that fieldnote, enter 'none'
    if not references[i]:
        references[i] = 'none'

URL not found: http://fieldnotes.makingandknowing.org/2018-Fall/Li-Verdigris_Field_Notes_FA18/Li-Verdigris_Field_Notes_FA18.html
URL not found: http://fieldnotes.makingandknowing.org/2018-Fall/Olson-Bread_Molding_Reconstruction_Field_Notes_FA18/Olson-Bread_Molding_Reconstruction_Field_Notes_FA18.html
URL not found: http://fieldnotes.makingandknowing.org/2018-Fall/Olson-Printmaking_Techniques_Field_Notes/Olson-Printmaking_Techniques_Field_Notes.html
URL not found: http://fieldnotes.makingandknowing.org/2018-Fall/Olson-Verdigris_Field_Notes_FA18/Olson-Verdigris_Field_Notes_FA18.html
URL not found: http://fieldnotes.makingandknowing.org/2018-Fall/Rosenkranz-Verdigris_EXAMPLE_Field_Notes_FA18/Rosenkranz-Verdigris_EXAMPLE_Field_Notes_FA18.html


In [7]:
fieldnotes.insert(len(fieldnotes.columns), df_refs_column, references) # add the references as a column at the end

In [8]:
fieldnotes.to_csv(out_path, index=False)